In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

def generate_data(sample_size, corr, mean=2000, sigma=300):
    """Генерируем коррелированные данные исходной метрики и ковариаты.
    
    sample_size - размер выборки
    corr - корреляция исходной метрики с ковариатой
    mean - среднее значение исходной метрики
    sigma - стандартное отклонение исходной метрики

    return - pd.DataFrame со столбцами ['metric', 'covariate'],
        'metric' - значения исходной метрики,
        'covariate' - значения ковариаты.
    """
    means = np.array([mean, mean])
    cov = sigma ** 2 * np.array([[1, corr], [corr, 1]])
    data = np.random.multivariate_normal(means, cov, sample_size).astype(int)
    df = pd.DataFrame({'metric': data[:, 0], 'covariate': data[:, 1]})
    return df

def calculate_theta(metrics, covariates):
    """Вычисляем Theta.

    metrics - значения исходной метрики
    covariates - значения ковариаты

    return - theta.
    """
    covariance = np.cov(covariates, metrics)[0, 1]
    variance = covariates.var()
    theta = covariance / variance
    return theta

def check_ttest(df_control, df_pilot):
    """Проверяет гипотезу о равенстве средних с помощью t-test.

    return - pvalue.
    """
    values_control = df_control['metric'].values
    values_pilot = df_pilot['metric'].values
    _, pvalue = stats.ttest_ind(values_control, values_pilot)
    return pvalue

def check_cuped(df_control, df_pilot, df_theta):
    """Проверяет гипотезу о равенстве средних с использованием CUPED.
    
    df_control и df_pilot - данные контрольной и экспериментальной групп
    df_theta - данные для оценки theta

    return - pvalue.
    """
    theta = calculate_theta(df_theta['metric'], df_theta['covariate'])
    metric_cuped_control = df_control['metric'] - theta * df_control['covariate']
    metric_cuped_pilot = df_pilot['metric'] - theta * df_pilot['covariate']
    lift = (metric_cuped_pilot.mean()-metric_cuped_control.mean())/metric_cuped_control.mean()
    print(lift)
    _, pvalue = stats.ttest_ind(metric_cuped_control, metric_cuped_pilot)
    return pvalue

### Вариант 1

1. Используем формулу Y_cuped = Y - (X - AVG(X)) * COV(Y, X)/VAR(X)
2. Считаем theta на всем наборе данных

In [2]:
sample_size = 100000      # размер групп
corr = 0.6             # корреляция ковариаты с целевой метрикой
effect = 0             # размер эффекта

pvalues = []

for i in range(1000):
    df_control = generate_data(sample_size, corr)
    df_pilot = generate_data(sample_size, corr)
    df_pilot['metric'] += effect
    df_theta = pd.concat([df_control, df_pilot])
    theta = calculate_theta(df_theta['metric'], df_theta['covariate'])
    metric_cuped_control = df_control['metric'] - theta * (df_control['covariate']-df_control['covariate'].mean())
    metric_cuped_pilot = df_pilot['metric'] - theta * (df_pilot['covariate']-df_pilot['covariate'].mean())
    lift_cuped = (metric_cuped_pilot.mean()-metric_cuped_control.mean())/metric_cuped_control.mean()
    lift_orig = (df_pilot['metric'].mean() - df_control['metric'].mean())/df_control['metric'].mean()
    # variance reduction
    vr_c = (df_control["metric"].std()**2-metric_cuped_control.std()**2)/df_control["metric"].std()**2
    vr_t = (df_pilot["metric"].std()**2-metric_cuped_pilot.std()**2)/df_pilot["metric"].std()**2
    #print(f"Lift (original): {lift_orig}, Lift (CUPED): {lift_cuped}")
    #print(f"""Variance reduction (control): {vr_c}, 
    #          Variance reduction (test): {vr_t}""")

    _, pvalue_orig = stats.ttest_ind(df_control['metric'], df_pilot['metric'])
    _, pvalue_cuped = stats.ttest_ind(metric_cuped_control, metric_cuped_pilot)

    #print(f"pvalue (original): {pvalue_orig}, pvalue (CUPED): {pvalue_cuped}")
    pvalues.append([pvalue_orig, pvalue_cuped, lift_orig, lift_cuped])

In [3]:
i_type_error_orig = sum([1 for i in pvalues if i[0]<0.05])/len(pvalues)
i_type_error_cuped = sum([1 for i in pvalues if i[1]<0.05])/len(pvalues)
print(f"""I type error (original): {i_type_error_orig}, 
I type error (CUPED): {i_type_error_cuped}""")
print(f"Max lift difference: {max([(i[2]-i[3]) for i in pvalues])}")
print(f"Min lift difference: {min([(i[2]-i[3]) for i in pvalues])}")

I type error (original): 0.054, 
I type error (CUPED): 0.115
Max lift difference: 1.4417635845709819e-12
Min lift difference: -7.096230748198165e-13


### Вариант 2

1. Используем формулу Y_cuped = Y - X * COV(Y, X)/VAR(X)
2. Считаем theta на всем наборе данных

In [4]:
pvalues = []

for i in range(1000):
    df_control = generate_data(sample_size, corr)
    df_pilot = generate_data(sample_size, corr)
    df_pilot['metric'] += effect
    df_theta = pd.concat([df_control, df_pilot])
    theta = calculate_theta(df_theta['metric'], df_theta['covariate'])
    metric_cuped_control = df_control['metric'] - theta * (df_control['covariate'])
    metric_cuped_pilot = df_pilot['metric'] - theta * (df_pilot['covariate'])
    lift_cuped = (metric_cuped_pilot.mean()-metric_cuped_control.mean())/metric_cuped_control.mean()
    lift_orig = (df_pilot['metric'].mean() - df_control['metric'].mean())/df_control['metric'].mean()
    # variance reduction
    vr_c = (df_control["metric"].std()**2-metric_cuped_control.std()**2)/df_control["metric"].std()**2
    vr_t = (df_pilot["metric"].std()**2-metric_cuped_pilot.std()**2)/df_pilot["metric"].std()**2
    #print(f"Lift (original): {lift_orig}, Lift (CUPED): {lift_cuped}")
    #print(f"""Variance reduction (control): {vr_c}, 
    #          Variance reduction (test): {vr_t}""")

    _, pvalue_orig = stats.ttest_ind(df_control['metric'], df_pilot['metric'])
    _, pvalue_cuped = stats.ttest_ind(metric_cuped_control, metric_cuped_pilot)

    #print(f"pvalue (original): {pvalue_orig}, pvalue (CUPED): {pvalue_cuped}")
    pvalues.append([pvalue_orig, pvalue_cuped, lift_orig, lift_cuped])

In [5]:
i_type_error_orig = sum([1 for i in pvalues if i[0]<0.05])/len(pvalues)
i_type_error_cuped = sum([1 for i in pvalues if i[1]<0.05])/len(pvalues)
print(f"""I type error (original): {i_type_error_orig}, 
I type error (CUPED): {i_type_error_cuped}""")
print(f"Max lift difference: {max([(i[2]-i[3]) for i in pvalues])}")
print(f"Min lift difference: {min([(i[2]-i[3]) for i in pvalues])}")

I type error (original): 0.054, 
I type error (CUPED): 0.058
Max lift difference: 0.0027958929785924817
Min lift difference: -0.0031904183082200814


### Вариант 3

1. Используем формулу Y_cuped = Y - X * COV(Y, X)/VAR(X)
2. Считаем theta на каждой группе отдельно

In [6]:
pvalues = []

for i in range(1000):
    df_control = generate_data(sample_size, corr)
    df_pilot = generate_data(sample_size, corr)
    df_pilot['metric'] += effect
    df_theta = pd.concat([df_control, df_pilot])
    theta_control = calculate_theta(df_control['metric'], df_control['covariate'])
    theta_pilot = calculate_theta(df_pilot['metric'], df_pilot['covariate'])
    metric_cuped_control = df_control['metric'] - theta_control * (df_control['covariate']-df_control['covariate'].mean())
    metric_cuped_pilot = df_pilot['metric'] - theta_pilot * (df_pilot['covariate']-df_pilot['covariate'].mean())
    lift_cuped = (metric_cuped_pilot.mean()-metric_cuped_control.mean())/metric_cuped_control.mean()
    lift_orig = (df_pilot['metric'].mean() - df_control['metric'].mean())/df_control['metric'].mean()
    # variance reduction
    vr_c = (df_control["metric"].std()**2-metric_cuped_control.std()**2)/df_control["metric"].std()**2
    vr_t = (df_pilot["metric"].std()**2-metric_cuped_pilot.std()**2)/df_pilot["metric"].std()**2
    #print(f"Lift (original): {lift_orig}, Lift (CUPED): {lift_cuped}")
    #print(f"""Variance reduction (control): {vr_c}, 
    #          Variance reduction (test): {vr_t}""")

    _, pvalue_orig = stats.ttest_ind(df_control['metric'], df_pilot['metric'])
    _, pvalue_cuped = stats.ttest_ind(metric_cuped_control, metric_cuped_pilot)

    #print(f"pvalue (original): {pvalue_orig}, pvalue (CUPED): {pvalue_cuped}")
    pvalues.append([pvalue_orig, pvalue_cuped, lift_orig, lift_cuped])

In [7]:
i_type_error_orig = sum([1 for i in pvalues if i[0]<0.05])/len(pvalues)
i_type_error_cuped = sum([1 for i in pvalues if i[1]<0.05])/len(pvalues)
print(f"""I type error (original): {i_type_error_orig}, 
I type error (CUPED): {i_type_error_cuped}""")
print(f"Max lift difference: {max([(i[2]-i[3]) for i in pvalues])}")
print(f"Min lift difference: {min([(i[2]-i[3]) for i in pvalues])}")

I type error (original): 0.047, 
I type error (CUPED): 0.115
Max lift difference: 1.74691597992771e-12
Min lift difference: -3.9090887644921413e-13
